# Cargamos las librerías

In [37]:
import metodos_wt as mwt
import nltk
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
from lmd_dicts import get_sentiment_dicts
from ipywidgets import interact_manual, fixed
import ipywidgets as widgets
%load_ext autoreload
%autoreload 2
plt.style.use('fivethirtyeight')
style = {'description_width': 'initial'}
layout = widgets.Layout(width='400px')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Cargamos la data: Correr solo al principio

In [2]:
db_path = 'C:\\Users\\ASUS\\larrainvial.com\\Equipo Quant - Documentos\\Area Estrategias Cuantitativas 2.0\\BDD\\Data Alternativa'
o_path = db_path
transcript_dict = mwt.load_obj(db_path + '\\transcript_bbd')
count_dict = {}

# Transformamos los transcripts en bags of words

In [10]:
# Cargamos acá las palabras de los clusters ya que nos sirve para no eliminar estos términos
cluster_words = pd.read_excel(db_path + '\\transcript_clusters.xlsx')
keep_w = list(cluster_words.stack().values)

# Cargamos las herramientas necesarias. Si quisiéramos tener más SW las agregamos acá
stemmer = nltk.stem.SnowballStemmer('english')
sw = nltk.corpus.stopwords.words('english')
sw_extra = ['&', 'call', 'also', 'year', 'and', 'we', 'so', 'this']
sw += sw_extra
vectorizer = CountVectorizer(stop_words='english', ngram_range=(1, 2))

for key, data in transcript_dict.items():
    # Chequeamos que no esté vacío
    if data.shape[0] == 0:
        continue
    # Feedback de cómo va el proceso
    print(key)
    
    # Algunos ajustes
    data['date'] = pd.to_datetime(data['date'])
    data = data.loc[data['type'] == 'Earnings Call']
    data = data[['date', 'text']].set_index('date').squeeze()

    # Si hay un solo transcript, pasamos
    if not isinstance(data, pd.Series):
        continue

    cv_dict = {}
    for date, text in data.iteritems():
        if not isinstance(text, str):
            continue
        text = mwt.process_raw_text(text, sw, stemmer)
        cv_dict[date] = mwt.count_vect(text, vectorizer, keep_w=keep_w)

    df = pd.DataFrame(cv_dict).fillna(0)
    cols_orden = df.columns.sort_values()
    df = df[cols_orden]
    if df.shape[0] > 0:
        count_dict[key.split(':')[-1]] = df

# Creamos un df con la última data disponible que será útil para el análisis general
last_dict =  pd.DataFrame({c: df.iloc[:, -1] for c, df in count_dict.items() if not df.empty})
last_delta = pd.DataFrame({c: df.diff().iloc[:, -1] for c, df in count_dict.items() if not df.empty})

NasdaqGS:MELI
BOVESPA:VALE3
BOVESPA:PETR4
BMV:AMX L
BMV:WALMEX *
BOVESPA:ITUB4
BOVESPA:ABEV3
NYSE:SCCO
BOVESPA:BBDC4
BOVESPA:WEGE3
BOVESPA:MGLU3
NasdaqGS:XP
BMV:GMEXICO B
BOVESPA:SANB11
BOVESPA:B3SA3
BVC:ECOPETROL
BMV:FEMSA UBD
NASDAQ:STNE
BOVESPA:BPAC11
BOVESPA:BPAC5
BOVESPA:BBAS3
BOVESPA:ITSA4
BOVESPA:VIVT4
LSE:ANTO
NYSE:PAGS
BOVESPA:NTCO3
BMV:GFNORTE O
BOVESPA:SUZB3
BOVESPA:JBSS3
SNSE:COPEC
SNSE:ENELAM
BOVESPA:BTOW3
NYSE:BAP
SNSE:SQM-B
BOVESPA:HAPV3
BOVESPA:ELET6
BOVESPA:BBSE3
BMV:BIMBO A
BOVESPA:LAME4
NYSE:KOF
BOVESPA:RENT3
NYSE:GLOB
BOVESPA:RADL3
BMV:CUERVO *
SNSE:CHILE
BOVESPA:CRFB3
BMV:AC *
SNSE:FALABELLA
BASE:TECO2
SNSE:BSANTANDER
BOVESPA:GGBR4
BOVESPA:RAIL3
BVC:ISA
BVC:BCOLOMBIA
BVC:GEB
BOVESPA:EGIE3
BMV:CEMEX CPO
BVC:BOGOTA
BOVESPA:CPFE3
BOVESPA:SBSP3
BOVESPA:LREN3
BOVESPA:VVAR3
SNSE:CMPC
BMV:BSMX B
BVC:GRUPOAVAL
BMV:IENOVA *
BMV:GCARSO A1
BOVESPA:KLBN11
SNSE:ENELCHILE
BOVESPA:CCRO3
BOVESPA:CSAN3
BMV:GRUMA B
SNSE:BCI
BMV:GAP B
BOVESPA:BRDT3
SNSE:CENCOSUD
BMV:LIVEPOL C-1
BMV:T

IndexError: single positional indexer is out-of-bounds

# Hacemos el conteo de palabras por tópico y además el sentiment
Podría hacerse en el for anterior, pero es probable que necesitemos normalizar por la suma asi que si usamos ese dato tiene que hacerse después.

In [16]:
# Primero separamos topics(clusters) y sentiment, ya que en sentiment crearemos la variable "delta".
# Además, funciona un poco distinto el método por eficiencia computacional (son muchas palabras)

clusters_dict = {}
sent_dict, negation_list = get_sentiment_dicts()
sentiment_dict = {}

for c in cluster_words.columns:
    words = cluster_words[c].values[cluster_words[c].notna()]
    cluster_dict = {}
    for k, df in count_dict.items():
        words_t = list(set(words) & set(df.index))
        subset = df.loc[words_t].sum()
        subset.index = [mwt.date_to_q(d) for d in subset.index]
        cluster_dict[k] = subset[~subset.index.duplicated(keep='first')]
        # Análisis de tono
        if c == cluster_words.columns[0]:
            sentiment_dict[k] = mwt.get_sentiment(df, sent_dict,
                                                  negation_list)
    clusters_dict[c] = pd.DataFrame(cluster_dict)

# Cambiamos el formato de la evolución de sentiment al de los tópicos y juntamos ambos diccionarios
sentiments = list(next(iter(sentiment_dict.values())).columns)
sentiment_dict = {s: pd.DataFrame({k: df[s] for k, df in sentiment_dict.items()})
                  for s in sentiments}
clusters_dict = {**sentiment_dict, **clusters_dict}

# Análisis del universo

### Análisis de las top y bottom empresas por cluster

In [27]:
w = interact_manual(mwt.get_top_clusters,
                    clusters_dict=fixed(clusters_dict),
                    cluster=widgets.Dropdown(options=clusters_dict.keys(), value='Risk', description='Cluster:'),
                    N=widgets.IntSlider(value=10, description='N: ', max=30, min=5),
                    cambio=widgets.Checkbox(True, description='Medir cambio:'))

interactive(children=(Dropdown(description='Cluster:', index=7, options=('Negative', 'Positive', 'Delta', 'Deb…

### Resumen por industria

### Empresas con más cambios c/r al transcipt pasado

In [30]:
w = interact_manual(mwt.top_changes,
                    delta_df=fixed(last_delta),
                    N=widgets.IntSlider(value=10, description='N: ', max=30, min=5))

interactive(children=(IntSlider(value=10, description='N: ', max=30, min=5), Button(description='Run Interact'…

# Análisis de industria

### Scatter cluster A vs cluster B

### Evolución clusters

### Contribución por palabras

# Análisis de empresa específica

### Gráfico Winning Eleven vs universo e industria

### Contribución por palabras a clusters

In [46]:
w = interact_manual(mwt.cluster_contribution,
                    last_df=fixed(last_df),
                    company=widgets.Text(value='', description='Company'),
                    cluster_words=fixed(cluster_words),
                    N=widgets.IntSlider(value=10, description='# de palabras: ', max=30, min=5, style=style))

interactive(children=(Text(value='', description='Company'), IntSlider(value=10, description='# de palabras: '…

### Evolución Sentiment

### Mayores cambios por palabra

In [38]:
w = interact_manual(mwt.main_changes,
                    count_dict=fixed(count_dict),
                    company=widgets.Text(value='', description='Company'),
                    MM_per=widgets.IntSlider(value=4, description='# de quarters: ', max=10, min=1, style=style),
                    N=widgets.IntSlider(value=10, description='# de palabras: ', max=30, min=5, style=style))

interactive(children=(Text(value='', description='Company'), IntSlider(value=4, description='# de quarters: ',…

# Cell de prueba

In [40]:
for i in range(7):
    print(i%2, int(i/2)*2)

0 0
1 0
0 2
1 2
0 4
1 4
0 6


In [28]:
last_delta = pd.DataFrame({c: df.diff().iloc[:, -1] for c, df in count_dict.items() if not df.empty})